# Đề thi mẫu chứng chỉ TensorFlow - Dạng bài 03

```
ProtonX - TensorFlow Class 
```
Giới thiệu về chứng chỉ - https://www.tensorflow.org/certificate

> Chú ý: đọc kỹ các nội dung trong Handle Notebook trong liên kết trên.

---

### Hướng dẫn làm bài 

Đọc kỹ File `Tensorflow Exam - Tips` trên hệ thống Học tập.

*Một số yêu cầu:
- Sử dụng TensorFlow 2.x trở lên.

Phân loại ảnh thực tế:
- Một số loại dataset thường gặp trong đề thi: Chó - mèo, Ngựa - người, Dao - Búa - Kéo.
- Có thể đề yêu cầu load data từ tf.dataset. Đề bài có thể yêu cầu resize ảnh về kích thước nào đó (ví dụ 224x224x3) và re-scale input về khoảng [-1, 1]. Thực hiện resize và rescale ảnh trong hàm map:
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map

- Lưu ý 1: Có thể sử dụng Transfer learning - ưu tiên MobileNet để Train mô hình có độ chính xác cao hơn.
- Lưu ý 2: Phân loại nhị phân nhưng đề bài có thể yêu cầu lớp cuối cùng của model phải sử dụng hàm softmax(thay vì sigmoid), vì vậy cần chọn làm loss phù hợp với hàm softmax.
- flow_from_directory + generator
  + Chó mèo: https://youtu.be/kShvYHDXvvg
  + Ngựa người: https://youtu.be/ee9tF9xEf04
  + Dao búa kéo: https://www.tensorflow.org/datasets/catalog/rock_paper_scissors
  + Cách resize ảnh, rescale input và transfer learning có thể tham khảo: https://www.tensorflow.org/tutorials/images/transfer_learning
https://youtu.be/Y-4KLFt_c6Y

- Bài sử dụng transfer learning có thể có kết quả khi submit là 2/5, 3/5, …. Vì vậy nên submit nhiều lần. Nếu đạt từ 4/5 trở lên thì có thể làm bài khác (áp dụng với tất cả các bài)

- Chạy lại toàn bộ File này trước khi ấn nộp bài.

### Đề thi mẫu bài 03

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)

/home/danieldu/anaconda3/envs/protonx/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.8.0


In [2]:
# Hoặc tải dữ liệu trong TensorFlow Dataset
DATASET_NAME = 'rock_paper_scissors'
IMG_SIZE = 300
INP_SHAPE = (IMG_SIZE,IMG_SIZE, 3)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE
(ds_train, ds_test), ds_info = tfds.load(
    DATASET_NAME,
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

2023-10-11 20:29:13.591400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-11 20:29:13.591605: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-11 20:29:13.617611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-11 20:29:13.617803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-11 20:29:13.617952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [3]:
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomContrast(0.2)
])

def prepare(ds, shuffle=False, augment=False):
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), num_parallel_calls=AUTOTUNE)
  if shuffle:
    ds = ds.shuffle(1000)
  ds = ds.batch(batch_size)
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
  return ds.prefetch(buffer_size=AUTOTUNE)

In [4]:
ds_train = prepare(ds_train, shuffle=True, augment=True)
ds_test = prepare(ds_test)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=INP_SHAPE),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [6]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
model.fit(
    ds_train,
    epochs=100,
    validation_data=ds_test,
    batch_size=32
)

Epoch 1/100


/home/danieldu/anaconda3/envs/protonx/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-10-11 20:29:16.569717: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8500


79/79 [==============================] - 9s 86ms/step - loss: 1.2174 - accuracy: 0.3508 - val_loss: 1.0977 - val_accuracy: 0.3871
Epoch 2/100
79/79 [==============================] - 7s 80ms/step - loss: 1.0380 - accuracy: 0.4623 - val_loss: 1.0187 - val_accuracy: 0.6613
Epoch 3/100
79/79 [==============================] - 8s 94ms/step - loss: 0.8850 - accuracy: 0.5762 - val_loss: 0.8465 - val_accuracy: 0.6613
Epoch 4/100
79/79 [==============================] - 7s 79ms/step - loss: 0.7297 - accuracy: 0.6671 - val_loss: 0.6843 - val_accuracy: 0.7151
Epoch 5/100
79/79 [==============================] - 7s 79ms/step - loss: 0.5872 - accuracy: 0.7409 - val_loss: 0.4937 - val_accuracy: 0.8763
Epoch 6/100
79/79 [==============================] - 7s 83ms/step - loss: 0.4963 - accuracy: 0.7968 - val_loss: 0.4342 - val_accuracy: 0.8548
Epoch 7/100
79/79 [==============================] - 7s 83ms/step - loss: 0.4376 - accuracy: 0.8119 - val_loss: 0.4136 - val_accuracy: 0.8522
Epoch 8/100
79/79 

In [7]:
# Lưu Model và nộp 
model.save("mymodel.h5")